In [ ]:
import nltk
import numpy as np
import requests
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from nltk.corpus import treebank, brown, conll2000
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Part-of-Speech Tagging with a Bidirectional LSTM

It's difficult to find free sequence labelling datasets because they're so labour-intensive to create.
<br><br>
Fortunately, **Natural Language Toolkit (NLTK)** includes enough free sets of labelled corpora for our purposes. NLTK also provides them in a convenient uniform format.<br>
https://www.nltk.org/index.html<br>
https://www.nltk.org/nltk_data/<br>
<br>
We'll use the Treebank, Brown, and CONLL-2000 datasets.

In [ ]:
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')

In their original form, the datasets use different part-of-speech (PoS) tag sets. We need to ensure they all use the same tagset, so we'll download a simplified set called the *universal_tagset* from NLTK.<br>

See Section 2.3 here for a list of tags: https://www.nltk.org/book/ch05.html

In [ ]:
nltk.download('universal_tagset')

We'll then retrieve the tagged sentences from each dataset, taking care to specify they should use the *universal tagset* we just downloaded. We'll then combine them into one collection.

In [ ]:
# Download all PoS-tagged sentences and place them in one list.
tagged_sentences = treebank.tagged_sents(tagset='universal') +\
                   brown.tagged_sents(tagset='universal') +\
                   conll2000.tagged_sents(tagset='universal')

print(tagged_sentences[0])
print(f"Dataset size: {len(tagged_sentences)}")

Each tagged sentence is actually a list of word-tag tuples (bear in mind that NLTK's universal tagset is a reduced tagset so items such as *proper nouns* are simply tagged as *nouns*).<br>

Our model is going to take in a sequence of words, and output a sequence of PoS tags, so we need to separate the words from the tags in our dataset. The tag sequences will serve as our training labels.

In [ ]:
sentences, sentence_tags = [], []

for s in tagged_sentences:
  sentence, tags = zip(*s)
  sentences.append(list(sentence))
  sentence_tags.append(list(tags))

The sentences and their respective tags are now in separate lists.

In [ ]:
print(sentences[0])
print(sentence_tags[0])

In [ ]:
print(len(sentences), len(sentence_tags))

Create train/validation/test splits. This time, we don't have a separate test set so we'll call *train_test_split* twice.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

x_train, x_test, y_train, y_test = train_test_split(sentences, sentence_tags,
                                                    test_size=1 - train_ratio,
                                                    random_state=1)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test,
                                                test_size=test_ratio/(test_ratio + validation_ratio),
                                                random_state=1)

In [ ]:
print(len(x_train), len(y_train))
print(len(x_val), len(y_val))
print(len(x_test), len(y_test))

Now that we have our datasets preprocessed, the next step is to vectorize. 

First, we need to create a tokenizer for the sentences and *fit* it to the training dataset to create a vocabulary. We'll just use the default tokenizer settings which applies some light filtering, lowers the case, and separates on spaces. We'll also supply an out-of-vocabulary token (\<OOV\>) in case the tokenizer encounters words during testing/inference which it doesn't during training.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
sentence_tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<OOV>')

In [ ]:
sentence_tokenizer.fit_on_texts(x_train)

In [ ]:
print(f"Vocabulary size: {len(sentence_tokenizer.word_index)}")

We also need to create *another* tokenizer for the tags since our labels are also sequences. This time, we won't need an OOV token because there are only a handful of tags and, in this case, they'll all be encountered during training.

In [ ]:
tag_tokenizer = keras.preprocessing.text.Tokenizer()
tag_tokenizer.fit_on_texts(y_train)

In [ ]:
print(f"Number of PoS tags: {len(tag_tokenizer.word_index)}\n")
tag_tokenizer.get_config()

In [ ]:
# The set of universal PoS tags.
tag_tokenizer.word_index

Next, we need to vectorize our sentences and corresponding tags, we'll use the tokenizer's *texts_to_sequences* method to convert each sentence to a sequence of integers where each integer maps to a particular token.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences

In [ ]:
x_train_seqs = sentence_tokenizer.texts_to_sequences(x_train)

In [ ]:
print(x_train_seqs[0])

We can use the *sequences_to_texts* method to convert a vectorized sentence back to its preprocessed form.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#sequences_to_texts

In [ ]:
print(f"Original: {x_train[0]}")
print(f"Reconstructed: {sentence_tokenizer.sequences_to_texts([x_train_seqs[0]])}")

Next, we'll vectorize the labels (i.e. sequences of PoS tags) using its respective tokenizer.

In [ ]:
y_train_seqs = tag_tokenizer.texts_to_sequences(y_train)

In [ ]:
tag_tokenizer.sequences_to_texts([y_train_seqs[0]])

Finally, we'll do the same with the validation inputs and labels.

In [ ]:
x_val_seqs = sentence_tokenizer.texts_to_sequences(x_val)
y_val_seqs = tag_tokenizer.texts_to_sequences(y_val)

As we covered in the slides, **Recurrent Neural Networks** are capable of handling variable length sequences.<br><br>
Despite that, it's still best to pad or truncate sequences to a uniform length for one or both of these reasons:<br>
1. Performance. The longer a sequence, the higher the computation cost. One may want to truncate long sequences to a shorter length if that's feasible and doesn't result in too much performance loss.

2. When processing datasets in batches, each sequence *in a batch* usually has to be of uniform length.<br>

For simplicity, here, we'll make *every* sequence be as long as the longest sequence. In other words, we'll determine how long the longest sequence is, then pad out the rest of the sequences to be the same length.<br>

A more optimized solution would be to make each sequence as long as the longest sequence in each *batch* to avoid unnecessary processing.

In [ ]:
MAX_LENGTH = len(max(x_train_seqs, key=len))
print(f"Length of longest input sequence: {MAX_LENGTH}")

We can pad the sequences with the *pad_sequences* method:<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [ ]:
x_train_padded = keras.preprocessing.sequence.pad_sequences(x_train_seqs, padding='post',
                                                            maxlen=MAX_LENGTH)

In [ ]:
print(x_train_padded[0])

We'll do the same with the training label (PoS sequences)...

In [ ]:
y_train_padded = keras.preprocessing.sequence.pad_sequences(y_train_seqs, padding='post',
                                                            maxlen=MAX_LENGTH)

...and the validation dataset.

In [ ]:
x_val_padded = keras.preprocessing.sequence.pad_sequences(x_val_seqs, padding='post', maxlen=MAX_LENGTH)
y_val_padded = keras.preprocessing.sequence.pad_sequences(y_val_seqs, padding='post', maxlen=MAX_LENGTH)

PoS tagging is a multiclass classification task done at each timestep, so we need to convert every tag for every sentence into a one-hot encoding<br>
https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical<br>

In [ ]:
y_train_categoricals = keras.utils.to_categorical(y_train_padded)

The label (PoS tag sequence) for a single sentence is now a **sequence of one-hot encodings**. These will serve as our training targets.

In [ ]:
# The one hot encodings for the first label.
print(y_train_categoricals[0])

In [ ]:
# One-hot encoding for a single tag.
print(y_train_categoricals[0][0])

We can determine the PoS tag from a one-hot encoding by seeing which index is set to 1, then using that to query the tag tokenizer's *index_word* dictionary.

In [ ]:
idx = np.argmax(y_train_categoricals[0][0])
print(f"Index: {idx}")

print(f"Tag: {tag_tokenizer.index_word[idx]}")

We'll one-hot encode the validation labels as well.

In [ ]:
y_val_categoricals = keras.utils.to_categorical(y_val_padded)

At this point, we're ready to build our model. We'll train word embeddings concurrently with our model 
There are several new things here:<br>
1. The embedding layer has a *mask_zero* parameter. We added padding in order to make our batches the same size, but we don't want the model to make PoS predictions on padding. Setting *mask_zero* to *True* makes the layers following the embedding layer ignore padding values.<br>
https://www.tensorflow.org/guide/keras/masking_and_padding<br>
https://stackoverflow.com/questions/47485216/how-does-mask-zero-in-keras-embedding-layer-work<br><br>
2. We're using a **bidirectional LSTM**. The *Bidirectional* layer is a wrapper to which we pass an *LSTM* layer. The first parameter to the *LSTM* layer is the number of units in the cell. The second parameter, *return_sequences*, controls whether the RNN returns an output for each timestep or only the last output. Since we're doing PoS-tagging, we want an output for each timestep and so *return_sequences* is set to *True*.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM<br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional


In [ ]:
# For the embedding layer. "+ 1" to account for the padding token.
num_tokens = len(sentence_tokenizer.word_index) + 1
embedding_dim = 128

# For the output layer. The number of classes corresponds to the
# number of possible tags.
num_classes = len(tag_tokenizer.word_index) + 1

In [ ]:
# The set_seed call and kernel_initializer parameters are used here to
# ensure you and I get the same results. To get random weight initializations,
# remove them.
tf.random.set_seed(0)

model = keras.Sequential()

model.add(layers.Embedding(input_dim=num_tokens,
                           output_dim=embedding_dim,
                           input_length=MAX_LENGTH,
                           mask_zero=True))

model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True,
                                           kernel_initializer=tf.keras.initializers.random_normal(seed=1))))

model.add(layers.Dense(num_classes, activation='softmax',
                       kernel_initializer=tf.keras.initializers.random_normal(seed=1)))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


A few notes about the model summary:<br>

The embedding layer **output** has three dimensions:
- Batch size (it's showing as "None" because we didn't specify it upfront. We'll do it when we call *model.fit*).
- Sequence length (the sequences are all the same length now after our padding step).
- Embedding dimension.
<br><br>

The LSTM outputs a vector *twice* the size of what we specified because it's bidirectional. Recall from the slides that the outputs from the two LSTMs will be concatenated before going to the output layer.
<br><br>

The final layer's **output** also has three dimensions:
- Batch size
- Sequence length
- Output dimension (the number of possible tags).

The output will be a **sequence of probability distributions** for each input sequence. One probability distribution per tag.



In [ ]:
model.summary()

we'll use *early stopping* with some *patience* to halt training once validation loss stops improving.<br>

The model will compare its output (sequences of softmax-generated probability distributions) against the one-hot encoded targets.



In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train_padded, y_train_categoricals, epochs=20,
                    batch_size=256, validation_data=(x_val_padded, y_val_categoricals),
                    callbacks=[es_callback])

Once our model is trained, we'll vectorize and pad the testing dataset. In the case of the labels, we'll also one-hot encode them.

In [ ]:
# Preprocess the test data and test the model.
x_test_seqs = sentence_tokenizer.texts_to_sequences(x_test)
x_test_padded = keras.preprocessing.sequence.pad_sequences(x_test_seqs, padding='post', maxlen=MAX_LENGTH)

y_test_seqs = tag_tokenizer.texts_to_sequences(y_test)
y_test_padded = keras.preprocessing.sequence.pad_sequences(y_test_seqs, padding='post', maxlen=MAX_LENGTH)
y_test_categoricals = keras.utils.to_categorical(y_test_padded)

In [ ]:
model.evaluate(x_test_padded, y_test_categoricals)

We can now use our model to tag sentences.

In [ ]:
samples = [
    "Brown refused to testify.",
    "Brown sofas are on sale.",
]

The function below takes a list of strings, tokenizes and pads them, then has the model tag them. Note that if a sentence is longer than MAX_LENGTH, it'll be truncated.

In [ ]:
def tag_sentences(sentences):
  sentences_seqs = sentence_tokenizer.texts_to_sequences(sentences)
  sentences_padded = keras.preprocessing.sequence.pad_sequences(sentences_seqs,
                                                                maxlen=MAX_LENGTH,
                                                                padding='post')

  # The model returns a LIST of PROBABILITY DISTRIBUTIONS (due to the softmax)
  # for EACH sentence. There is one probability distribution for each PoS tag.
  tag_preds = model.predict(sentences_padded)

  sentence_tags = []

  # For EACH LIST of probability distributions...
  for i, preds in enumerate(tag_preds):

    # Extract the most probable tag from EACH probability distribution.
    # Note how we're extracting tags for only the non-padding tokens.
    tags_seq = [np.argmax(p) for p in preds[:len(sentences_seqs[i])]]

    # Convert the sentence and tag sequences back to their token counterparts.
    words = [sentence_tokenizer.index_word[w] for w in sentences_seqs[i]]
    tags = [tag_tokenizer.index_word[t] for t in tags_seq]
    sentence_tags.append(list(zip(words, tags)))

  return sentence_tags


In [ ]:
tagged_sample_sentences = tag_sentences(samples)

In [ ]:
print(tagged_sample_sentences[0])

In [ ]:
print(tagged_sample_sentences[1])

So that's one way to build a PoS tagger. Industrial-strength taggers use a lot more data and these days, are powered by more sophisticated models which we'll learn about when we cover *transformers*.